In [336]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load CSV
batting = pd.read_csv(r"C:\Users\HP\Downloads\ipl_batting.csv")


STEP 2: Create batting summary

In [337]:
summary = batting.groupby('batsmen').agg(
    runs=('R', 'sum'),
    balls=('B', 'sum'),
    fours=('fours', 'sum'),
    sixes=('sixes', 'sum')
).reset_index()


STEP 3: Calculate OUTS & AVERAGE

In [338]:
outs = batting[batting['wicket_status'] == 'out'].groupby('batsmen').size()
summary['outs'] = summary['batsmen'].map(outs).fillna(0)

summary['average'] = summary['runs'] / summary['outs']
summary.loc[summary['outs'] == 0, 'average'] = summary['runs']



 STEP 4: Calculate 100s (Centuries)

In [339]:
hundreds = batting.groupby(['match_no', 'batsmen'])['R'].sum().reset_index()
hundreds = hundreds[hundreds['R'] >= 100]
hundreds_count = hundreds.groupby('batsmen').size()

summary['hundreds'] = summary['batsmen'].map(hundreds_count).fillna(0)


STEP 5: Take TOP 10 batsmen by runs

In [340]:
top = summary.sort_values(by='runs', ascending=False).head(10)


📊 STEP 6: PLOTLY DASHBOARD (2×2)

In [341]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Runs by Batsmen",
        "Fours & Sixes by Batsmen",
        "Batting Average",
        "Centuries (100s)"
    )
)

# 1️⃣ Runs by batsmen (Area chart)
fig.add_trace(
    go.Scatter(
        x=top['batsmen'],
        y=top['runs'],
        fill='tozeroy',
        mode='lines+markers',
        name='Runs'
    ),
    row=1, col=1
)

# 2️⃣ Fours & Sixes (Grouped Bar)
fig.add_trace(
    go.Bar(
        x=top['batsmen'],
        y=top['fours'],
        name='Fours'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=top['batsmen'],
        y=top['sixes'],
        name='Sixes'
    ),
    row=1, col=2
)

# 3️⃣ Batting Average
fig.add_trace(
    go.Bar(
        x=top['batsmen'],
        y=top['average'],
        name='Average'
    ),
    row=2, col=1
)

# 4️⃣ Centuries
fig.add_trace(
    go.Bar(
        x=top['batsmen'],
        y=top['hundreds'],
        name='100s'
    ),
    row=2, col=2
)
fig.update_layout(
    title=dict(
        text="IPL Batting Analysis Dashboard",
        x=0.5,
        font=dict(size=22)
    ),
    autosize=True,
    height=900,
    width=779,
    margin=dict(l=30, r=30, t=80, b=30),
    barmode='group',
    template='plotly_dark'
)


fig.show()
